<a href="https://colab.research.google.com/github/hoseinm444/FanapExercises/blob/main/telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install themoviedb
#!pip install python-telegram-bot
#!pip install python-telegram-bot-pagination
#!pip install requests

import json
from themoviedb import TMDb
from typing import Final
import requests
from telegram_bot_pagination import InlineKeyboardPaginator
from telegram import Update ,KeyboardButton,InlineKeyboardButton,InlineKeyboardMarkup ,ReplyKeyboardMarkup,Bot
from telegram.ext import Application , CommandHandler,CallbackQueryHandler , MessageHandler, ContextTypes , filters
import nest_asyncio
nest_asyncio.apply()

token:Final = ''
Bot_username:Final = ''
tmdb = TMDb(key="", language="En", region="En")


#command
#start command
async def start_command(update: Update,context:ContextTypes.DEFAULT_TYPE):
  #when user hit start our response is so:
  print("start button..")#for debugging
  await update.message.reply_text(text="Welcome to YourBot! Please click on search button to search for your desire movie name...")

#help command
async def help_command(update: Update,context:ContextTypes.DEFAULT_TYPE):
  #tell user how the bot works :
  print("help button..") #for debugging
  await update.message.reply_text("clik on search button , then enter your desire movie name and we will show you at most 100 result movies according to it."

#search command
async def search_command(update: Update,context:ContextTypes.DEFAULT_TYPE) :
    print("search button..\n") #for debugging
    await update.message.reply_text("Please Enter your movie name :")

def search_for_movie(text:str):
    processed : str =text.lower()
    print(f'search for : {processed}')#for debugging
    items = tmdb.search().movies(processed)
    movie_range = items.total_results
    dictionary = {}
    i=1
    try:
      while i<=5:                                                         #items.total_pages
        items = tmdb.search().movies(processed,page=i)
        print(f'Movie : {items}\n')
        print(f'Movie Itmes : {items.total_results} , total_Pages:{items.total_pages},current_page:{items.page}\n')
        i +=1
        for j in items:
          movie_id =j.id
          movie = tmdb.movie(movie_id).details(append_to_response="credits,external_ids,images,videos")
          #movie_format = json.dumps(movie, default=str)
          dictionary.__setitem__(movie.id, movie.title)
    except Exception as e:
               print(f'error:{e}')
    #print(len(dictionary))
           #movie = tmdb.movie(movie_id).details(append_to_response="credits,external_ids,images,videos"
           #dictionary.__setitem__(movie.id, movie.title)
    return dictionary
    if(items.total_results == 0):
      return 'we can not find such movie or your movie name is incorrect..'

async def handle_message(update: Update,context:ContextTypes.DEFAULT_TYPE):
    print('resopnse  is :')
    processed : str = update.message.text
    items = tmdb.search().movies(processed)
    dictionary = search_for_movie(processed)
    await update.message.reply_text(f"total results is :{items.total_results}  we can show 100 numbers of them becuse of telegram limitations .")
    print(f'length of dictionary{len(dictionary)}')
    i=0
    if len(dictionary)>0:
      for key , value in dictionary.items():
           keyboard = [
                    [
                         InlineKeyboardButton("جزئیات", callback_data=f'{key}'),
                         #InlineKeyboardButton("imdb url", callback_data=f'{value}'),
                    ],
                 ]
           i +=1
           reply_markup = InlineKeyboardMarkup(keyboard)
           #paginator = InlineKeyboardPaginator(len(dictionary),data_pattern='movie#{page}')
           #await update.message.reply_text(text=value,reply_markup=paginator.markup,parse_mode='Markdown')
           await update.message.reply_text(f"{i}-{value}:", reply_markup=reply_markup)
    else :
            await update.message.reply_text('your movie name is not found ')

async def button(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query
    print(f'query :{query}')
    print(f'query :{type(query.data)}')
    try:
      id_query_int = int(query.data)
    except error as e:
      print(e)
    movie = tmdb.movie(id_query_int).details(append_to_response="people,credits,external_ids,images,videos,person")
    gener_list =[]
    companies_list= []
    main_character=[]
    countries_list =[]
    for i in movie.credits.cast:
      main_character.append(i.name)
    #print(main_character)
    for i in movie.production_companies:
       companies_list.append(i.name)
    for i in movie.genres:
      gener_list.append(i.name)
    for i in movie.production_countries:
      countries_list.append(i.name)
    movie_dict =[
                  f'title : {movie.title}',
                  f'year : {movie.year}' ,
                  f'movie type  : {movie.media_type}' ,
                  f'popularity : {movie.popularity}' ,
                  f'imdb url : {movie.imdb_url}' ,
                  f'vote : {movie.vote_average}',
                  f'overview : {movie.overview}' ,
                  f'genres :{gener_list}',
                  f'companies : {companies_list}',
                  f'countries : {countries_list}',
                  f'characters :{main_character}' #make the text too long
               ]
    nl = '\n'
    print(movie_dict)
    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    await query.answer()

    await query.edit_message_text(text=f"movie is : {nl}{nl.join(movie_dict)}")

#logging errors
async def error(update: Update,context:ContextTypes.DEFAULT_TYPE):
  print('error....')
  print(f'Update {update} caused error {context.error}')
  #await update.message.reply_text(f'Update {update} caused error {context.error}')


if __name__ == '__main__' :
   # this for debugging
   print('starting bot...')
   app = Application.builder().token(token).build()

   # Buttoms


   # Commands
   app.add_handler(CommandHandler('start',start_command))
   app.add_handler(CommandHandler('help',help_command))
   app.add_handler(CommandHandler('search',search_command))
   app.add_handler(CallbackQueryHandler(button))

   # Messages
   app.add_handler(MessageHandler(filters.TEXT,handle_message))

   # Errors
   app.add_error_handler(error)
     # check for new message constantly  - polls the bot
   print('Polling....')
   app.run_polling(poll_interval=10)

starting bot...
Polling....
resopnse  is :
search for : yang
Movie : Movies(results=[PartialMovie(id=1098786, poster_path='/voV8720y0q99VdZn1cdu8D10mqP.jpg', adult=False, overview='Plot Unknown', release_date=datetime.date(1983, 12, 29), genre_ids=[18], original_title='Yang', original_language='id', title='My Dear', backdrop_path=None, popularity=1.031, vote_count=0, video=False, vote_average=0.0, media_type=<MediaType.movie: 'movie'>), PartialMovie(id=444282, poster_path='/oZay3PzHrJwccVlEkG9UyiMAB5B.jpg', adult=False, overview='A woman attempts a prayer circle to rid a household of a witch’s dark curse, but an encounter with evil will forever leave its mark on her.', release_date=datetime.date(2015, 12, 1), genre_ids=[27, 14], original_title='Yang', original_language='en', title='Yang', backdrop_path=None, popularity=0.6, vote_count=1, video=False, vote_average=9.0, media_type=<MediaType.movie: 'movie'>), PartialMovie(id=1107979, poster_path='/dLsAyZFzx37W1JUqd11JUl4KpZj.jpg', adult=

RuntimeError: Cannot close a running event loop